In [2]:
import pandas as pd
import numpy as np
from sklearn.cluster import MiniBatchKMeans
from MatrixFactorization import MatrixFactorization
from CollaborativeFiltering import CollaborativeFiltering as cb

In [3]:
movie_ratings = pd.read_csv('./dataset/ratings.csv')
movie_ratings_small = pd.read_csv('./dataset/ratings_small.csv')
movies_metadata = pd.read_csv('./dataset/movies_metadata.csv')

<ipython-input-3-0a9433d6ba92>:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_metadata = pd.read_csv('./dataset/movies_metadata.csv')


In [4]:
movie_ratings_small.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [5]:
movie_ratings.shape, movie_ratings_small.shape

((26024289, 4), (100004, 4))

In [6]:
movies_metadata = movies_metadata.drop_duplicates(subset='id', keep='first')

In [7]:
movies_metadata = movies_metadata[movies_metadata['id'].str.isdigit()]
movies_metadata['id'] = movies_metadata['id'].astype('int64')

<ipython-input-7-7420a33fdfa0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_metadata['id'] = movies_metadata['id'].astype('int64')


In [8]:
# 두 데이터프레임 병합 (movieId 기준)
movies_total= movies_metadata.merge(movie_ratings,
                                        left_on='id',
                                        right_on='movieId',
                                        how='left')

# 필요 없어진 movieId 열 제거
movies_total = movies_total.drop(columns=['movieId'])


In [9]:
movies_total.shape

(11474436, 27)

In [10]:
movies_total.dropna(subset='userId', inplace=True)

In [11]:
movies_total['userId'] = movies_total['userId'].astype('int64')

In [12]:
a =movies_total[['id','original_title','userId', 'rating']]
a.head()

,id,original_title,userId,rating
0,862,Toy Story,1923,3.0
1,862,Toy Story,2103,5.0
2,862,Toy Story,5380,1.0
3,862,Toy Story,6177,4.0
4,862,Toy Story,6525,4.0


In [13]:
user_matrix = pd.pivot_table(a, index="userId", columns='original_title',values='rating',aggfunc="mean", fill_value=0)
user_matrix

original_title,!Women Art Revolution,$5 a Day,'Gator Bait,'R Xmas,'Twas the Night Before Christmas,(A)Sexual,...And the Pursuit of Happiness,...Più forte ragazzi!,...altrimenti ci arrabbiamo!,00 Schneider - Jagd auf Nihil Baxter,...,친절한 금자씨,태풍,파란대문,하울링,해안선,헬로우 고스트,형사 Duelist,활,회사원,２０世紀少年< 第1章> 終わりの始まり
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270892,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
270893,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
270894,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
k = 2
kmeans = MiniBatchKMeans(n_clusters=k, random_state=42, batch_size=500)
item_clusters = kmeans.fit_predict(user_matrix.T)
unique_clusters = np.unique(item_clusters)
item_id = '장화, 홍련'
item_cluster_map = pd.Series(item_clusters, index=user_matrix.T.index)

# 특정 item_id의 군집 찾기
cluster_id = item_cluster_map.loc[item_id]
print(f"'{item_id}'은(는) {cluster_id}번 군집에 속합니다.")

# top_k = 10

cluster_indices = np.where(item_clusters == cluster_id)[0]  # 해당 클러스터 사용자 인덱스
cluster_matrix = user_matrix.T.iloc[cluster_indices]


'장화, 홍련'은(는) 0번 군집에 속합니다.


In [15]:
cluster_matrix.shape

(7461, 265917)

In [ ]:
mf = MatrixFactorization(cluster_matrix)
k_values = [50, 100, 200]  # 실험할 k 값 리스트
for k in k_values:
    actual_items, predicted_items = mf.predict(k, 10, False)
    precision = mf.precision_at_k(actual_items, predicted_items)
    print(f"k={k}, Precision: {precision:.4f}")

In [ ]:
mf = MatrixFactorization(cluster_matrix)
k_values = [300, 400, 500]  # 실험할 k 값 리스트
for k in k_values:
    actual_items, predicted_items = mf.predict(k, 10, False)
    precision = mf.precision_at_k(actual_items, predicted_items)
    print(f"k={k}, Precision: {precision:.4f}")

In [ ]:
k_values = [100, 200, 300, 400,500]  # 실험할 k 값 리스트
for k in k_values:
    actual_items, predicted_items = mf.predict(k, 10, False)
    recall = mf.recall_at_k(actual_items, predicted_items)
    print(f"k={k}, Recall: {recall:.4f}")

k=100, Recall: 0.8054
k=200, Recall: 0.8491
k=300, Recall: 0.8579
k=400, Recall: 0.8599
k=500, Recall: 0.8605


In [16]:
optimal_k = 100
cb_2 = cb(cluster_matrix, optimal_k)
item_title = "장화, 홍련"
idx = cluster_matrix.index.get_loc(item_title)
similarities = cb_2.adjusted_cosine_similarity()

In [17]:
similarities

<7461x7461 sparse matrix of type '<class 'numpy.float64'>'
	with 55666521 stored elements in Compressed Sparse Row format>

In [18]:
user_id = 1
ratings = cb_2.item_based_predict(idx, user_id)
print(f"사용자 {user_id}가 '{item_id}'에 부여할 예측 평점: {ratings}")

사용자 1가 '장화, 홍련'에 부여할 예측 평점: 4.209907743128816
